Import of libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler # To standardize the data

In [2]:
!pip install cvxpy
import cvxpy as cp

Import of the HMLasso function

In [3]:
### Adapt the path "C:/Users/Kilian/Desktop/ENSAE/STATAPP" to run the cell

import sys
sys.path.insert(1, 'C:/Users/Kilian/Desktop/ENSAE/STATAPP/Projet_Statapp/pretreatment')

import file_04_HMLasso as hml

## Data downloading and separation of the dataset

Dataset containing the types of each column from data_03.csv

In [21]:
columns_types = pd.read_csv("data_03_columns_types.csv", index_col=0)
columns_types.head(3)

,Name,Type
0,HHIDPN,Cont
1,HHID,Char
2,PN,Char


Downloading the data with social and genetic variables.

In [38]:
data = pd.read_csv("data_03.csv")

C:\Users\Kilian\AppData\Local\Temp\ipykernel_16096\3952530018.py:1: DtypeWarning: Columns (2684) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data_03.csv")


The column "genetic_Section_A_or_E" have mixed types, so we change its format.

In [39]:
temporary = np.where(data['genetic_Section_A_or_E'] == 'E', 1, np.where(data['genetic_Section_A_or_E'] == 'A', 0, np.nan))

In [40]:
data["genetic_Section_A_or_E"] = temporary

Now we add the health index created by t-SNE

In [50]:
tSNE_GHI = pd.read_csv("data_tSNE_GHI.csv")

We drop the one individual who have NA values for the age variables (Already deleted in the data_03)

In [51]:
tSNE_GHI.drop(tSNE_GHI[tSNE_GHI.HHIDPN == 32570030].index)

,HHIDPN,tSNE_GHI1,tSNE_GHI2,tSNE_GHI3,tSNE_GHI4,tSNE_GHI5,tSNE_GHI6,tSNE_GHI7,tSNE_GHI8,tSNE_GHI9,tSNE_GHI10,tSNE_GHI11,tSNE_GHI12,tSNE_GHI13,tSNE_GHI14
0,1010,46.338680,68.969490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,64.833760,-13.297101,27.734835,43.220688,44.066593,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3010,22.931510,-66.686300,-29.234972,13.921358,-91.249150,-44.982487,-73.210106,-93.987110,-26.113197,-49.452682,-13.593910,NaN,NaN,NaN
3,3020,87.220850,26.626232,-3.540929,-35.344840,5.839192,15.979321,87.244750,7.659104,7.514696,11.224237,6.606269,85.520730,NaN,NaN
4,10001010,16.348322,51.236960,-34.807380,16.018581,-20.836510,-49.169600,-41.571840,-27.421490,104.275900,-33.672630,-10.305906,-51.743675,-60.904327,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42228,923525020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.438065,39.893333,44.080963,52.331947,38.305832
42229,952836010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-40.590910,-29.552444,-54.016690,-32.491695,-30.162340
42230,958361010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.411686,51.926926,-52.184353,50.893330,NaN
42231,959733010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.747810,NaN,NaN,NaN,NaN


We sort tSNE_GHI and data by HHIDPN (indentifier)

In [52]:
tSNE_GHI.sort_values(by="HHIDPN", ascending=1)
data.sort_values(by="HHIDPN", ascending=1)

,HHIDPN,R1MPART,R2MPART,R3MPART,R4MPART,R5MPART,R6MPART,R7MPART,R8MPART,R9MPART,...,R13IADL5H_2,R13IADL5H_3,R13IADL5H_4,R13IADL5H_5,R14IADL5H_0,R14IADL5H_1,R14IADL5H_2,R14IADL5H_3,R14IADL5H_4,R14IADL5H_5
0,1010,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2010,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,3010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,3020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,10001010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42227,923525020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
42228,952836010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
42229,958361010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
42230,959733010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [49]:
tSNE_GHI

,HHIDPN,tSNE_GHI1,tSNE_GHI2,tSNE_GHI3,tSNE_GHI4,tSNE_GHI5,tSNE_GHI6,tSNE_GHI7,tSNE_GHI8,tSNE_GHI9,tSNE_GHI10,tSNE_GHI11,tSNE_GHI12,tSNE_GHI13,tSNE_GHI14
0,1010,46.338680,68.969490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,64.833760,-13.297101,27.734835,43.220688,44.066593,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3010,22.931510,-66.686300,-29.234972,13.921358,-91.249150,-44.982487,-73.210106,-93.987110,-26.113197,-49.452682,-13.593910,NaN,NaN,NaN
3,3020,87.220850,26.626232,-3.540929,-35.344840,5.839192,15.979321,87.244750,7.659104,7.514696,11.224237,6.606269,85.520730,NaN,NaN
4,10001010,16.348322,51.236960,-34.807380,16.018581,-20.836510,-49.169600,-41.571840,-27.421490,104.275900,-33.672630,-10.305906,-51.743675,-60.904327,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42228,923525020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.438065,39.893333,44.080963,52.331947,38.305832
42229,952836010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-40.590910,-29.552444,-54.016690,-32.491695,-30.162340
42230,958361010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.411686,51.926926,-52.184353,50.893330,NaN
42231,959733010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.747810,NaN,NaN,NaN,NaN
